In [1]:
import logging
import sys

!pip install -q google-generativeai

In [2]:
import google.generativeai as genai
from google.colab import userdata
import os
from dotenv import load_dotenv
try:
# Učitaj .env
        load_dotenv()
        GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Google Generative AI library installed and API key configured.")
except Exception as e:
        print(f"Error configuring Google API key: {e}")
        print("Please ensure 'GOOGLE_API_KEY' is set in Colab Secrets.")


Google Generative AI library installed and API key configured.


In [3]:
import google.generativeai as genai

def student_model(prompt: str) -> str:

        print(f"Student model received prompt: {prompt}")
        student_llm = genai.GenerativeModel('gemini-2.5-flash')

        # Generate content using the student model
        try:
            response = student_llm.generate_content(prompt)
            if response.text:
                solution = response.text
            else:
                solution = "No solution generated by student model."
        except Exception as e:
            solution = f"Error generating solution: {e}"
            print(solution)

        print(f"Student's solution: {solution}")
        return solution

print("Student model (Gemini Flash) redefined.")


Student model (Gemini Flash) redefined.


In [4]:
def professor_model(student_solution: str, current_prompt: str) -> str:

        print(f"Professor model received solution: {student_solution}\nfor prompt: {current_prompt}")

        # Initialize the stronger Gemini model for the professor
        professor_llm = genai.GenerativeModel('gemini-2.5-pro') # Using a 'stronger' model for the professor

        professor_prompt_text = f"""You are an expert prompt refiner. Your goal is to analyze a student's solution to a given prompt and provide a refined prompt that would lead to a better, more detailed, or more accurate solution in the next iteration.

Here is the current prompt:
`{current_prompt}`

Here is the student's solution based on that prompt:
`{student_solution}`

Based on the student's solution, identify areas where the original prompt could be improved. Think about what information was missing, ambiguous, or what constraints were not clear enough for the student to provide an optimal solution. Your refined prompt should guide the student towards a better answer by:
- Adding clarity or specificity.
- Introducing new constraints or considerations.
- Suggesting a different approach or focus.
- Asking for more detail or specific examples.

Your output MUST be ONLY the refined prompt string. Do not include any conversational text, prefixes like 'Refined Prompt:', or any other formatting. Just the new prompt string.
"""

        # Generate the refined prompt using the professor model
        try:
            response = professor_llm.generate_content(professor_prompt_text)
            if response.text:
                next_refined_prompt = response.text.strip()
            else:
                next_refined_prompt = current_prompt # Fallback if no text is generated
                print("Professor model did not return a refined prompt, using current prompt as fallback.")
        except Exception as e:
            next_refined_prompt = current_prompt # Fallback on error
            print(f"Error generating refined prompt with Professor model: {e}. Using current prompt as fallback.")

        print(f"\nProfessor's Feedback (Verbose):\n")
        print(f"Original Prompt: {current_prompt}")
        print(f"Student's Solution: {student_solution}")
        print(f"Refined Prompt for next iteration: {next_refined_prompt}")

        return next_refined_prompt

print("Professor model (Gemini Pro) redefined for intelligent prompt refinement.")


Professor model (Gemini Pro) redefined for intelligent prompt refinement.


In [6]:
prompt_history = []
student_solution_history = []

initial_prompt = "Zamišljaj svijet u kojem biljke mogu komunicirati međusobno putem svjetlosnih signala. Neke biljke su sporije, neke brže, a okolišni uvjeti poput temperature i vlažnosti utječu na brzinu njihove komunikacije. Zadatak: Osmisli sustav koji koordinira ovu biljnu mrežu kako bi važne informacije stigle od jedne strane šume do druge što brže, uzimajući u obzir ograničenja svjetlosnog signala i različite karakteristike biljaka."
prompt_history.append(initial_prompt)
print(f"Initial Prompt appended to history: {initial_prompt}")

# First student solution
student_initial_solution = student_model(initial_prompt)
student_solution_history.append(student_initial_solution)
print(f"\nInitial Student Solution: {student_initial_solution}")

# First iteration: Professor evaluates initial student solution
# The professor_model now directly returns the refined prompt
refined_prompt_1 = professor_model(student_initial_solution, initial_prompt)

prompt_history.append(refined_prompt_1)

print(f"\nProfessor's Feedback (Returned Refined Prompt - Iteration 1):\n{refined_prompt_1}")

num_additional_iterations = 4  # As specified in the previous notebook cell
current_prompt = refined_prompt_1

for i in range(num_additional_iterations):
    iteration_number = i + 2  # Start iteration numbering from 2
    print(f"\n--- Iteration {iteration_number} ---")

    # Student generates a solution based on the current prompt
    student_solution = student_model(current_prompt)
    student_solution_history.append(student_solution)
    print(f"\nStudent's Solution (Iteration {iteration_number}):\n{student_solution}")

    # Professor evaluates the solution and returns a refined prompt
    next_refined_prompt = professor_model(student_solution, current_prompt)
    prompt_history.append(next_refined_prompt)
    print(f"\nProfessor's Feedback (Returned Refined Prompt - Iteration {iteration_number}):\n{next_refined_prompt}")

    # Update current_prompt for the next iteration
    current_prompt = next_refined_prompt

print("\nIterative prompt refinement loop completed.")

Initial Prompt appended to history: Zamišljaj svijet u kojem biljke mogu komunicirati međusobno putem svjetlosnih signala. Neke biljke su sporije, neke brže, a okolišni uvjeti poput temperature i vlažnosti utječu na brzinu njihove komunikacije. Zadatak: Osmisli sustav koji koordinira ovu biljnu mrežu kako bi važne informacije stigle od jedne strane šume do druge što brže, uzimajući u obzir ograničenja svjetlosnog signala i različite karakteristike biljaka.
Student model received prompt: Zamišljaj svijet u kojem biljke mogu komunicirati međusobno putem svjetlosnih signala. Neke biljke su sporije, neke brže, a okolišni uvjeti poput temperature i vlažnosti utječu na brzinu njihove komunikacije. Zadatak: Osmisli sustav koji koordinira ovu biljnu mrežu kako bi važne informacije stigle od jedne strane šume do druge što brže, uzimajući u obzir ograničenja svjetlosnog signala i različite karakteristike biljaka.
Student's solution: Zamišljanje svijeta u kojem biljke komuniciraju svjetlosnim sig

Error generating refined prompt with Professor model: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
Please retry in 34

Error generating refined prompt with Professor model: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
Please retry in 4.

Error generating refined prompt with Professor model: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
Please retry in 34

Error generating refined prompt with Professor model: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
Please retry in 7.

Error generating refined prompt with Professor model: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro
Please retry in 36